## Implementing Collaborative Filtering from scratch

### Objective:
The goal of this exercise is to guide you through the implementation of collaborative filtering using matrix factorization and stochastic gradient descent (SGD). By the end of this exercise, students will have a working collaborative filtering system that predicts user ratings for movies

### Step 1: Understanding Collaborative Filtering:
Description: Collaborative filtering is a technique used in recommendation systems that predicts user preferences based on past interactions. It relies on the idea that users with similar preferences in the past will likely have similar preferences in the future.

Task: Research the concept of collaborative filtering and explain the difference between user-based and item-based filtering.

Question: What are the advantages of matrix factorization in collaborative filtering?

### Step 2: Loading the Dataset

Description: The MovieLens dataset contains user ratings for different movies. We will use two files:
* ratings.csv - contains userId, movieId, and rating.
* movies.csv - contains movieId and title.

Task: Load the dataset into a Pandas DataFrame named ratings and movies and inspect the first few rows.

Hint: Use pd.read_csv() to read the files rating.csv and movies.csv.

Question: What are the key columns in the ratings.csv file, and why are they important?
 **userId, movieId, rating**

Task: define three variables:
* ratings_shape is the ratings.csv shape, 
* n_users is the number of unique users and 
* n_movies is the number unique movies

In [7]:
import pandas as pd

movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")
ratings.head()
movies.head()


# Ratings_shape
ratings_shape = ratings.shape
# Unique Numbers
n_users = ratings["userId"].nunique()
# Unique movies
n_movies = movies["movieId"].nunique()



### Step 3: Creating the User-Item Matrix

Description: We need to convert our dataset into a user-item matrix where rows represent users, columns represent movies, and values represent ratings.

Task: Use Pandas pivot() function to transform the ratings data. Fill the empty spaces (or nan) with 0.

Hine: type the code in the next cell: user_item_matrix = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

The Pandas pivot, will convert the dataframe into another dataframe where the row index is 'userId'. the column index is 'movieId' and the values of the dataframe is 'rating'. 
Think of the result dataframe as the matrix called user_item_matrix.

In [8]:
user_item_matrix = ratings.pivot(index="userId", columns="movieId", values="rating").fillna(0)

### Step 4: Initializing Parameters

Description: We need to initialize two matrices: U (user features) and M (movie features). These matrices represent latent (means hidden) factors that define user preferences and movie characteristics.

Task: Initialize U and M matrices randomly.

Remember: 
* n_users is the number of user
* n_movies is the number of movies
* the latent factors are parameter you can play with, set it to 10 (latents_factors=10)

The latent_factors variable determines the number of hidden features in the matrix factorization model. It represents the dimensions of user preferences and movie characteristics.

Effects:

* Too few factors → The model may not capture enough information, leading to poor recommendations.
* Too many factors → The model may overfit, capturing noise instead of meaningful patterns.

A reasonable number is chosen based on dataset size and experimentation.

In [9]:
import numpy as np

R = user_item_matrix.values
num_users, num_items = R.shape

print('R.shape=', R.shape, 'num_users=', num_users, 'num_movies=', num_items)

# Randomly initialize user and item matrices
latents_factors = 10 
# Initializing User Features
U = np.random.rand(num_users, latents_factors)
#Initializing Movie Features
M = np.random.rand(num_items, latents_factors)

# Print the shape of the matrices
print("Shape of U (user features):", U.shape)
print("Shape of M (movie features):", M.shape)

R.shape= (610, 9724) num_users= 610 num_movies= 9724
Shape of U (user features): (610, 10)
Shape of M (movie features): (9724, 10)


### Step 5: Implementing the Training Algorithm

Description: We will use Stochastic Gradient Descent (SGD) to update U and M based on observed ratings. The goal is to minimize the error between predicted and actual ratings.

The dot product between U (user preferences) and M (movie characteristics) represents the predicted rating a user would give to a movie. A higher value indicates a stronger preference, guiding the recommendation system.

Regularization is a term that we add during updating the matricies U and M to helps prevent overfitting by penalizing large values in U and M. This encourages the model to generalize better to unseen data.
* Without regularization: The model may memorize (overfitting) training data, leading to poor performance on new ratings.
* With regularization: The model balances fitting known ratings while maintaining generalization ability.

Task: Implement the SGD update rule for training the model. Type the code below:

```
Learning_rate = 0.01
reg_param = 0.02 # Reqgularization parameter to prevent overfitting
epochs = 100 # number of iterations

for epoch in range(epochs):
    for i in range(num_users): # loop on all users
        for j in range(num_movies): # loop on all movies
            if user_item_matrix.iloc[i, j] > 0:  # Only consider known ratings
                error = user_item_matrix.iloc[i, j] - np.dot(U[i, :], M[j, :].T) # Multiply matrix U[i,:] (specific user) and M[:, j] (specific movie)
                U[i, :] += learning_rate * (error * M[j, :] - reg_param * U[i, :]) # Update user latent_factors for specific user
                M[j, :] += learning_rate * (error * U[i, :] - reg_param * M[j, :]) # Update movie latent_factors for specific movie
    # compute the root mean square error between the actual user_item_matrix and the predicted matrix:
    mse = np.mean((user_item_matrix.values[user_item_matrix.values > 0] - np.dot(U, M.T)[user_item_matrix.values > 0]) ** 2)
    if (epoch+1)%20 == 0: print(f"Epoch {epoch + 1}, MSE: {mse:.4f}") # print root mean square error every 20 iterations
```
Updating U and M only for known ratings ensures that we learn from actual data rather than making arbitrary updates based on missing values. This prevents incorrect assumptions and improves model accuracy.

In [10]:
Learning_rate = 0.01
reg_param = 0.02 # Reqgularization parameter to prevent overfitting
epochs = 100 # number of iterations

for epoch in range(epochs):
    for i in range(num_users): # loop on all users
        for j in range(num_items): # loop on all movies
            if R[i, j] > 0:  # Only consider known ratings
                error = R[i, j] - np.dot(U[i, :], M[j, :].T) # Multiply matrix U[i,:] (specific user) and M[:, j] (specific movie)
                U[i, :] += Learning_rate * (error * M[j, :] - reg_param * U[i, :]) # Update user latent_factors for specific user
                M[j, :] += Learning_rate * (error * U[i, :] - reg_param * M[j, :]) # Update movie latent_factors for specific movie
    # compute the root mean square error between the actual user_item_matrix and the predicted matrix:
    mse = np.mean((R[R > 0] - np.dot(U, M.T)[R > 0]) ** 2)
    if (epoch+1) % 10 == 0: print(f"Epoch {epoch + 1}, MSE: {mse:.4f}") # print root mean square error every 20 iterations

Epoch 10, MSE: 0.6673
Epoch 20, MSE: 0.5490
Epoch 30, MSE: 0.4725
Epoch 40, MSE: 0.4270
Epoch 50, MSE: 0.3978
Epoch 60, MSE: 0.3776
Epoch 70, MSE: 0.3626
Epoch 80, MSE: 0.3512
Epoch 90, MSE: 0.3422
Epoch 100, MSE: 0.3349


### Step 6: Making Predictions

Description: Now that we have trained U and M, we can make predictions by computing the dot product of U and V.

Task: Implement a function to predict the rating for a specific user and movie. Type the code below:
```
def predict(user, movie): # predict the rating of a user to a movie.
    user_idx = user - 1  # Adjust index
    movie_idx = movie - 1
    # multiple matrix U and M for that specific user and specific movie
    predicted_rating = np.dot(U[user_idx, :], M[movie_idx, :].T)
    # Find the movie name:
    movie_name = movies.loc[movies['movieId'] == movie, 'title'].values[0]
    # Return the predict rating for that specific user and the movie name
    return predicted_rating, movie_name

prediction, movie_name = predict(1, 10)
print(f"Predicted rating for User 1 and Movie '{movie_name}': {prediction:.2f}")
```

In [11]:
def predict(user, movie): # predict the rating of a user to a movie.
    user_idx = user - 1  # Adjust index
    movie_idx = movie - 1
    # multiple matrix U and M for that specific user and specific movie
    predicted_rating = np.dot(U[user_idx, :], M[movie_idx, :].T)
    # Find the movie name:
    movie_name = movies.loc[movies['movieId'] == movie, 'title'].values[0]
    # Return the predict rating for that specific user and the movie name
    return predicted_rating, movie_name

# Predict rating for user 1 and movie 10
prediction, movie_name = predict(1, 10)
print(f"Predicted rating for User 1 and Movie '{movie_name}': {prediction:.2f}")

Predicted rating for User 1 and Movie 'GoldenEye (1995)': 4.06


### Step 7: Evaluating the Model

Description: Evaluate the performance of the model using Mean Squared Error (MSE) or other relevant metrics.

Task: Compute the overall error on the dataset.

In [15]:
# Write the code here
# Use the root mean squared error (RMSE) formula.

# Compute the predicted ratings
predicted_ratings = np.dot(U, M.T)
mask = R > 0
rmse = np.sqrt(np.mean((R[mask] - predicted_ratings[mask]) ** 2))
print(f"Final RMSE on known ratings: {rmse:.4f}")

Final RMSE on known ratings: 0.5787


Generate the top-N recommendations for a specific user. We need to compute the predicted ratings for all movies a user hasn't rated, then return the highest ranked movies.
Type in the following code:
```
def recommend_movies(user, N=5):
    user_idx = user - 1  # Adjust index
    predicted_ratings = np.dot(U[user_idx, :], M.T) # multiply the U for a specific user with all movies M latent_factors
    movie_indices = np.argsort(predicted_ratings)[::-1]  # Sort in descending order

    recommended_movies = []
    for movie_idx in movie_indices:
        movie_id = user_item_matrix.columns[movie_idx] # get the movie id
         if user_item_matrix.iloc[user, movie_idx] != 0:
            movie_name = movies.loc[movies['movieId'] == movie_id, 'title'].values[0] # get the movie name
            recommended_movies.append((movie_name, predicted_ratings[movie_idx]))  # append the movie name with predicted rating
        if len(recommended_movies) >= N: # stop when you collect N movies.
            break
    
    return recommended_movies

# Example: Get top 5 recommendations for User 1
top_movies = recommend_movies(1)
for movie, rating in top_movies:
    print(f"Movie: {movie}, Predicted Rating: {rating:.2f}")
```

In [16]:
def recommend_movies(user, N=5):
    user_idx = user - 1  # Adjust index
    predicted_ratings = np.dot(U[user_idx, :], M.T) # multiply the U for a specific user with all movies M latent_factors
    movie_indices = np.argsort(predicted_ratings)[::-1]  # Sort in descending order

    recommended_movies = []
    for movie_idx in movie_indices:
        movie_id = user_item_matrix.columns[movie_idx] # get the movie id
        if user_item_matrix.iloc[user, movie_idx] != 0:
            movie_name = movies.loc[movies['movieId'] == movie_id, 'title'].values[0] # get the movie name
            recommended_movies.append((movie_name, predicted_ratings[movie_idx]))  # append the movie name with predicted rating
        if len(recommended_movies) >= N: # stop when you collect N movies.
            break
    
    return recommended_movies

# Example: Get top 5 recommendations for User 1
top_movies = recommend_movies(1)
for movie, rating in top_movies:
    print(f"Movie: {movie}, Predicted Rating: {rating:.2f}")

Movie: The Jinx: The Life and Deaths of Robert Durst (2015), Predicted Rating: 6.46
Movie: Shawshank Redemption, The (1994), Predicted Rating: 5.71
Movie: Good Will Hunting (1997), Predicted Rating: 5.64
Movie: Exit Through the Gift Shop (2010), Predicted Rating: 5.41
Movie: Warrior (2011), Predicted Rating: 5.38


#### Submission Instructions

Answer all conceptual questions.

1. Submit the complete Python implementation with comments explaining each step.
2. Provide a short analysis of your model's performance and potential improvements.

Good luck!